In [2]:
import pandas as pd
from rapidfuzz import process, fuzz

## Prepare data

### Redispatch data

The redispatch data can be downloaded from https://www.netztransparenz.de/de-de/Systemdienstleistungen/Betriebsfuehrung/Redispatch. 

In [12]:
redispatch_data = pd.read_csv('data/input/redispatch_data.csv', encoding='utf-8', sep=';')
units = redispatch_data[['BETROFFENE_ANLAGE']].drop_duplicates()
units = units.reset_index(drop=True)
units.to_csv('data/input/redispatched_units.csv')

### Conventional plants

The list of conventional plants can be downloaded from https://data.open-power-system-data.org/conventional_power_plants/.

In [13]:
conventional_plants = pd.read_csv('data/input/conventional_power_plants_DE.csv')
conventional_plants = conventional_plants[['name_bnetza', 'city', 'state', 'lat', 'lon']].drop_duplicates()
conventional_plants = conventional_plants.reset_index(drop=True)
conventional_plants.to_csv('data/input/filtered_conventional.csv')

## Determine location coordinates 

### Fuzzy matching

We consider fuzzy matching. FuzzyWuzzy is a library that uses Levenshtein distance to calculate the differences between strings. The Levenstein distance between two strings is defined as the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one string into the other.

In [19]:
df_plants = pd.read_csv("data/input/filtered_conventional.csv")  
df_red_units = pd.read_csv( "data/input/redispatched_units.csv") 

We want to find a match based on 3 columns from the dataframe containing power plants information.

In [15]:
choices = {}
for col in ['name_bnetza', 'city', 'state']:
    choices[col] = df_plants[col].dropna().unique()

There are multiple possibilites to compare two strings, see, e.g., https://medium.com/@laxmi17sarki/string-matching-using-fuzzywuzzy-24be9e85c88d:
- Ratio: pure Levenstein distance calculation
- Partial ratio: considers subsets of the strings it compares, and then returns a ratio according to their similarities
- Token sort ratio: ignores case, punctuation and order of the words (tokens) in the string
- Weighted ratio: weights are applied based on the lengths of the strings

We use the weighted ratio.

In [20]:
def find_best_match(row, column_name, threshold):
    best_match = process.extractOne(row, choices[column_name], scorer=fuzz.WRatio)
    if best_match and best_match[1] > threshold: 
        return best_match[0], best_match[1]
    return None, float('-inf')

In [21]:
def match(threshold=70):
    matches = []
    for anlage in df_red_units['BETROFFENE_ANLAGE']:
        best_score, best_match, best_col = float('-inf'), None, None
        for col in ['name_bnetza', 'city', 'state']:
            match, score = find_best_match(anlage, col, threshold)
            if match:
                if score > best_score:
                    best_score = score
                    best_match = match
                    best_col = col
        matched_row = df_plants[df_plants[best_col] == best_match].iloc[0] if best_col else None
        matches.append({
            'BETROFFENE_ANLAGE': anlage,
            'Matched_Column': best_col,
            'Value': best_match,
            'Score': best_score,
            'Latitude': matched_row['lat'] if best_col else None,
            'Longitude': matched_row['lon'] if best_col else None
        })
    
    matched_df = pd.DataFrame(matches)
    return matched_df

We first set a threshold of 50.

In [22]:
threshold = 50
print(f'Total redispatched units: {len(df_red_units)}')

matched_df = match(threshold)
matched_df = matched_df.sort_values(by="Score", ascending=False).reset_index(drop=True)
matched_df.to_csv(f"data/output/fuzzy_matching_{threshold}.csv")
print(f'Matched redispatched units with threshold = {threshold}: {len(matched_df[~matched_df['Matched_Column'].isna()])}')

Total redispatched units: 338
Matched redispatched units with threshold = 50: 310


#### Analyze results

We analize the results and observe that the matches are not accurate as the score goes below ~88.

In [23]:
final_threshold = 87
filtered_matched_df = matched_df[matched_df['Score'] > final_threshold].reset_index(drop=True)
filtered_matched_df.to_csv("data/output/fuzzy_matching_final.csv")

unmatched_df = matched_df[matched_df['Score'] <= final_threshold].reset_index(drop=True)
unmatched_df.to_csv("data/output/fuzzy_unmatched.csv")

print(f'Matched units with threshold {final_threshold}: {len(filtered_matched_df)}/{len(df_red_units)}')
print(f'Unmatched units with threshold {final_threshold}: {len(unmatched_df)}/{len(df_red_units)}')


Matched units with threshold 87: 164/338
Unmatched units with threshold 87: 174/338


### Manual matching

In [24]:
manually_matched = pd.read_excel('data/output/manually_matched.xlsx')
print(f'Manually matched units: {len(manually_matched[~manually_matched['Value'].isna()])}/{len(unmatched_df)}')

Manually matched units: 122/174


### Final results

In [ ]:
fuzzy_matched = pd.read_csv('data/output/fuzzy_matching_final.csv')
fuzzy_matched = fuzzy_matched.drop(columns=['Score', 'Matched_Column'])
manually_match = pd.read_excel('data/output/manually_matched.xlsx')
combined_matched = pd.concat([fuzzy_matched, manually_match], ignore_index=True)

combined_matched.to_csv('data/output/geo_redispatched_units.csv')
combined_matched.to_excel('data/output/geo_redispatched_units.xlsx')

redispatch_data = redispatch_data.merge(combined_matched[['BETROFFENE_ANLAGE', 'Latitude', 'Longitude']], 
                on='BETROFFENE_ANLAGE', 
                how='left')
redispatch_data.to_csv('data/output/geo_redispatch_data.csv')